In [ ]:
#Instalação do pacote de otimização, principalmente para google colab
#!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!

In [1]:
import pandas as pd
from datetime import datetime, timedelta
#Pacote de otimização
from ortools.linear_solver import pywraplp
from ortools.linear_solver.pywraplp import Variable
#Pacote para data handling da otimização
from typing import Dict, Set, Tuple, Union, List, Any
from collections import defaultdict


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:

idIntegrador = "SSA"
venda: Dict[Tuple[str, str], Variable] = {}
inputPersonalizacao = {"nucleo":0,
         "granjas_grandes":['384','301','382'],#['802','291','332'],
         "dias_consec_nucleo_geral":3,
         "dias_consec_granja_nucleo":{},#381:3
         "lotes_em_dias_fixos":{}#{"79018":["2023-10-03","ITA"]}#"76902":["5/7/2023","ITA"],"76903":["8/7/2023","ITA"]
         }
# venda do lot, que esta na granja, no dia, do integrador (var principal)
venda_lote: Dict[Tuple[str, Tuple[str, str], datetime.date, str], Variable] = {}
# venda da granja, no dia, do integrador (var auxiliar)
venda_granja: Dict[Tuple[str, datetime.date, str], Variable] = {}
# venda do nucleo da granja, no dia
venda_nucleo: Dict[Tuple[Tuple[str, str], datetime.date, str], Variable] = {}
# quantidade de animais (para lote, granja, data e integrador)
n_animais: Dict[Tuple[str, Tuple[str, str], datetime.date, str], int] = {}
#qual a granja do lote
lot_to_granja: Dict[str, str] = {}
# qual o nucleo do lote
lot_to_nucleo: Dict[str, str] = {}
# idade do lote, com granja no dia t do integrador
aviario: Dict[Tuple[str, Tuple[str, str], datetime.date, str], str] = {}
# idade do lote, com granja no dia t do integrador
idade: Dict[Tuple[str, Tuple[str, str], datetime.date, str], int] = {}
# sexo do lote, com granja no dia t do integrador
nroLote: Dict[Tuple[str, Tuple[str, str], datetime.date, str], str] = {}
# sexo do lote, com granja no dia t do integrador
sexo: Dict[Tuple[str, Tuple[str, str], datetime.date, str], str] = {}
# km do lote, com granja no dia t do integrador
km: Dict[Tuple[str, Tuple[str, str], datetime.date, str], float] = {}
# consumo do lote, com granja no dia t do integrador
consumo: Dict[Tuple[str, Tuple[str, str], datetime.date, str], float] = {}
# peso do lote, com granja no dia t do integrador
peso: Dict[Tuple[str, Tuple[str, str], datetime.date, str], float] = {}
# uma lista de tempo para cada lote
tempos_by_lote: Dict[str, List[datetime.date]] = defaultdict(lambda: [])
# todos os lotes que existem
todos_lotes: Set[str] = set()
# todas as granjas que existem
granjas: Set[str] = set()
nucleos_by_granja: Dict[str, Set[str]] = defaultdict(lambda: set())
lotes_by_granjas: Dict[str, Set[str]] = defaultdict(lambda: set())
# todos os integradores que existem
#grupos de lotes machos e femeas
lotes_macho = set()
lotes_femea = set()
# lucro e custo para o lot, no dia, do integrador
novo_lucro: Dict[Tuple[str, Tuple[str, str], datetime.date, str], float] = {}
novo_custo: Dict[Tuple[str, Tuple[str, str], datetime.date, str], float] = {}

todos_tempos: Set[datetime.date] = set()
dias_parado: Dict[str,List[datetime.date]] = defaultdict(lambda: [])
dias_de_abate: Dict[str,List[datetime.date]] = defaultdict(lambda: [])

todos_abatedouros = list()
dados_abatedouro: Dict[Tuple[Any,Any], Any] = {}
global pesos_df
objective: str = "lucro"
solver = "SCIP"
solverParams = None

In [5]:
def config_solver(gap_limit: float = 1e-3, time_limit: int = 60000 ):
        global solver, solverParams
        #Solver SCIP
        # self.solver = pywraplp.Solver.CreateSolver(solver)
        #Solver CBC MIP
        solver = pywraplp.Solver('hello_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

        solver.EnableOutput()
        solver.SetTimeLimit(time_limit)


        solverParams = pywraplp.MPSolverParameters()
        solverParams.SetDoubleParam(solverParams.RELATIVE_MIP_GAP, gap_limit)


In [ ]:
## Realizando a leitura dos arquivos( no drive) e colocando no formato dataframe via google colab 
#list_pesos_df = pd.read_csv('/content/drive/MyDrive/Otimizador PLT/dados_lote_otim.csv', delimiter = ';')
#param_iniciais = pd.read_csv('/content/drive/MyDrive/Otimizador PLT/param_integradores.csv', delimiter = ',')

In [ ]:
## Realizando a leitura dos arquivos e colocando no formato dataframe

list_pesos_df = pd.read_csv('dados_lote_otim.csv', delimiter=';')
param_iniciais = pd.read_csv('param_integradores.csv', delimiter=',')

In [7]:
def get_values(pesos_df):
        global param_iniciais, todos_tempos, lotes_by_granjas, nroLote, aviario, novo_lucro, novo_custo, n_animais, idade, km, sexo, consumo, peso, granjas, todos_lotes, tempos_by_lote, todos_abatedouros, lot_to_nucleo
        # print(pesos_df)

        todos_abatedouros = list(set(param_iniciais["idIntegrador"]))
        for integrador in todos_abatedouros:
            for _, row in pesos_df.iterrows():
                # print(row)
                idGalpao = str(row.idGalpao)
                datetime_str = str(row.data_peso_predito)
                date = datetime.strptime(datetime_str, '%Y-%m-%d').date()
                granja = str(row.granja)
                nucleo = str(row.nucleo)
                key = idGalpao, (granja, nucleo), date, integrador
                nroLote[key] = str(row.nroLote)
                aviario[key] = str(row.aviario)
                novo_lucro[key] = float(row.lucro)
                novo_custo[key] = float(row.custo)
                n_animais[key] = int(row.qtde)
                idade[key] = int(row.day)
                km[key] = float(row.km)
                sexo[key] = str(row.sexo)
                consumo[key] = float(row.consumo)
                peso[key] = float(row.peso_predito)
                granjas.add(granja)
                lotes_by_granjas[granja].add(idGalpao)
                if idGalpao in lot_to_granja:
                    if not lot_to_granja[idGalpao] == granja:
                        raise ValueError(
                            f'em diferente data {date}, lote {idGalpao} esta em granja diferente {lot_to_granja[idGalpao]} e {granja}')
                else:
                    lot_to_granja[idGalpao] = granja

                nucleos_by_granja[granja].add(nucleo)
                if idGalpao in lot_to_nucleo:
                        if not lot_to_nucleo[idGalpao] == nucleo:
                            raise ValueError(
                                f'em diferente data {date}, lote {idGalpao} esta em nucleo diferente {lot_to_nucleo[idGalpao]} e {nucleo}')
                else:
                    lot_to_nucleo[idGalpao] = nucleo

                todos_lotes.add(idGalpao)
                todos_tempos.add(date)
                tempos_by_lote[idGalpao].append(date)
        todos_tempos = sorted(list(todos_tempos))

In [8]:
def get_params():
            global param_iniciais, dados_abatedouro,dias_parado,dias_de_abate
            dados_abatedouro = defaultdict(dict)
            for _, row in param_iniciais.iterrows():
                id_integrador = str(row.idIntegrador)
                aux_dia_semana = int(row.diaSemana)
                dia_semana = todos_tempos[aux_dia_semana]
                dia_parado = bool(row.diasParado)
                # print(dia_semana)

                if dia_parado:
                    dias_parado[id_integrador].append(dia_semana)
                else:
                    dias_de_abate[id_integrador].append(dia_semana)
                # print(dados_abatedouro)
                if id_integrador in todos_abatedouros:
                    # print(row)
                    # print(dados_abatedouro[(id_integrador,dia_semana)])['
                    # dados_abatedouro[id_integrador,dia_semana] = ParametrosIntegrador()
                    dados_abatedouro[(id_integrador,dia_semana)]['clientId']=str(row.clientId)
                    dados_abatedouro[(id_integrador,dia_semana)]['diaSemana']=int(row.diaSemana)
                    dados_abatedouro[(id_integrador,dia_semana)]['idIntegrador']=str(row.idIntegrador)
                    dados_abatedouro[(id_integrador,dia_semana)]['valorKgFrango']=float(row.valorKgFrango)
                    dados_abatedouro[(id_integrador,dia_semana)]['despesaAlimentarMedia']=float(row.despesaAlimentarMedia)
                    dados_abatedouro[(id_integrador,dia_semana)]['custoNaoAlimentarMedio']=float(row.custoNaoAlimentarMedio)
                    dados_abatedouro[(id_integrador,dia_semana)]['abateMaxDiario']=int(row.abateMaxDiario)
                    dados_abatedouro[(id_integrador,dia_semana)]['abateMinDiario']=int(row.abateMinDiario)
                    dados_abatedouro[(id_integrador,dia_semana)]['proporcaoMaxMachosFemeas']=float(row.proporcaoMaxMachosFemeas)
                    dados_abatedouro[(id_integrador,dia_semana)]['proporcaoMinMachosFemeas']=float(row.proporcaoMinMachosFemeas)
                    dados_abatedouro[(id_integrador,dia_semana)]['mediaMaxIdadeCiclo']=int(row.mediaMaxIdadeCiclo)
                    dados_abatedouro[(id_integrador,dia_semana)]['mediaMinIdadeCiclo']=int(row.mediaMinIdadeCiclo)
                    dados_abatedouro[(id_integrador,dia_semana)]['KmMedioMax']=float(row.KmMedioMax)
                    dados_abatedouro[(id_integrador,dia_semana)]['custoKm']=float(row.custoKm)
                    dados_abatedouro[(id_integrador,dia_semana)]['pesoCaixa']=float(row.pesoCaixa)
                    dados_abatedouro[(id_integrador,dia_semana)]['caixasCaminhao']=float(row.caixasCaminhao)
                    dados_abatedouro[(id_integrador,dia_semana)]['custoViagem']=float(row.custoViagem)
                    dados_abatedouro[(id_integrador,dia_semana)]['diasParado']=float(row.diasParado)


In [9]:
def set_variables_optimizer():
        global todos_lotes,lot_to_granja,lot_to_nucleo, todos_tempos, todos_abatedouros, venda_lote, venda_granja, venda_nucleo, solver, inputPersonalizacao
        # venda única
        for idGalpao in todos_lotes:
            granja = lot_to_granja[idGalpao]
            nucleo = lot_to_nucleo[idGalpao]
            for dia in todos_tempos:
                for integrador in todos_abatedouros:

                    #variavel principal
                    var_name = f"vendas_lote_{idGalpao}_{(granja, nucleo)}_{dia}_{integrador}"
                    venda_lote[idGalpao, (granja, nucleo), dia, integrador] = solver.IntVar(0, 1, var_name)

                    # variavel auxiliar para granja
                    key_granja = granja, dia , integrador
                    if key_granja not in venda_granja:
                        var_name = f"venda_granja_{granja}_{dia}_{integrador}"
                        venda_granja[key_granja] = solver.IntVar(0, 1, var_name)

                    # print(inputPersonalizacao)
                    # variavel auxiliar para nucleo
                    if inputPersonalizacao["nucleo"]:
                        key_nucleo = (granja, nucleo), dia , integrador
                        if key_nucleo not in venda_nucleo:
                            var_name = f"venda_granja_nucleo_{granja, nucleo}_{dia}_{integrador}"
                            venda_nucleo[key_nucleo] = solver.IntVar(0, 1, var_name)


In [10]:
def get_objective_value():
        global venda_lote, novo_lucro, objective, solver, n_animais
        print(objective)
        if objective=="lucro":

            func_objetivo = []
            for key in venda_lote:
                (idGalpao, (granja,nucleo), dia, integrador) = key
                qtd_animal = n_animais[key]
                lucro = novo_lucro[key]
                venda = venda_lote[key]
                func_objetivo.append((qtd_animal * lucro) * venda)

            solver.Maximize(solver.Sum(func_objetivo))

In [11]:
def venda_min_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                        todos_lotes, venda_minima,venda_lote):

        cstr_name = f'venda_min_para_dia_{dia}_{integrador}'
        # noinspection PyTypeChecker
        solver.Add(
            solver.Sum(
                [n_animais[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador]*
                    venda_lote[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador] for idGalpao in todos_lotes])
            >= venda_minima, cstr_name
            )


def venda_max_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                        todos_lotes, venda_maxima,venda_lote):

        cstr_name = f'venda_max_para_dia_{dia}_{integrador}'
        solver.Add(
            solver.Sum(
                [n_animais[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador] *
                    venda_lote[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador] for idGalpao in todos_lotes])
            <= venda_maxima,
            cstr_name
            )


In [12]:
def proporcao_maxima_lotes_macho_femea_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                        todos_lotes, sexo_lote,venda_lote, proporcao_max):
        solver.Add(
            solver.Sum(
                [n_animais[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador]*venda_lote[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador]
                for idGalpao in sexo_lote ] + [(-1 * float(proporcao_max))*n_animais[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador]*
                    venda_lote[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador]
                    for idGalpao in todos_lotes ])
                    <= 0
                    )

def proporcao_minima_lotes_macho_femea_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                        todos_lotes, sexo_lote,venda_lote, proporcao_min):
        solver.Add(
            solver.Sum(
                [n_animais[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador]*venda_lote[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador]
                for idGalpao in sexo_lote] + [(-1 * float(proporcao_min))*n_animais[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador]*
                    venda_lote[idGalpao, (lot_to_granja[idGalpao], lot_to_nucleo[idGalpao]), dia, integrador] for idGalpao in todos_lotes]
                ) >= 0
                )

In [13]:
def zerando_variaveis_dias_parado(solver, dias_parados, lote, granja, nucleo, integrador, venda_lote):

        for dia in dias_parados[integrador]:
            if dia in dias_parados[integrador]:
                cstr_name = f'l_{lote}_dias_parados_{dia}_{integrador}'
                solver.Add(venda_lote[lote, (granja, nucleo), dia, integrador] == 0, cstr_name)


In [14]:
def lotes_em_dias_fixos(solver,lotes_em_dias_fixos, todos_tempos, todos_abatedouros, idGalpao, lot_to_granja, lot_to_nucleo, venda_lote):
        lotes_fixos = lotes_em_dias_fixos[idGalpao]
        granja = lot_to_granja[idGalpao]
        nucleo = lot_to_nucleo[idGalpao]
        dia_str = lotes_fixos[0]
        dia_fixo = datetime.datetime.strptime(dia_str, '%Y-%m-%d').date()
        abatedouro_fixo = lotes_fixos[1]
        for dia in todos_tempos:
            for abatedouro in todos_abatedouros:
                if dia_fixo == dia and abatedouro_fixo == abatedouro:
                    cstr_name = f'lotes_dias_fixos_true_{idGalpao}_{dia_fixo}_{abatedouro_fixo}'
                    solver.Add(venda_lote[idGalpao, (granja, nucleo), dia_fixo, abatedouro_fixo] == 1, cstr_name)
                else:
                    cstr_name = f'lotes_dias_fixos_false_{idGalpao}_{dia}_{abatedouro}'
                    solver.Add(venda_lote[idGalpao, (granja, nucleo), dia, abatedouro] == 0, cstr_name)


In [15]:
def optimization_rules():
        global solver, tempos_by_lote, venda_lote, lotes_femea, lotes_macho, todos_lotes, lot_to_granja, lot_to_nucleo, todos_tempos, todos_abatedouros, venda_granja, venda_nucleo, n_animais, idade, sexo, inputPersonalizacao, dias_parado, dias_de_abate
        #%% Definindo as restrições
        for idGalpao, tempos in tempos_by_lote.items():
            # venda única
            cstr_name = f'venda_unica_{idGalpao}'
            granja = lot_to_granja[idGalpao]
            nucleo = lot_to_nucleo[idGalpao]
            solver.Add(
                solver.Sum(
                    [venda_lote[idGalpao, (granja, nucleo), dia, integrador] for dia in set(tempos) for integrador in todos_abatedouros]
                ) == 1, cstr_name)

            #Separando os lotes por sexo do frango
            for integrador in todos_abatedouros:
                for dia in set(tempos):
                    if sexo[(idGalpao, (granja, nucleo), dia, integrador)] == "f":
                        lotes_femea.add(idGalpao)
                    elif sexo[(idGalpao, (granja, nucleo), dia, integrador)]== "m":
                        lotes_macho.add(idGalpao)


        for key, var in venda_lote.items():
            idGalpao, _, dia, integrador = key
            # print(idade[key])
            if idade[key] < dados_abatedouro[(integrador,dia)]['mediaMinIdadeCiclo']:
                cstr_name = f'idade_min_l_{idGalpao}_dia_{dia}_abat_{integrador}'
                solver.Add(var == 0, cstr_name)
            if idade[key] > dados_abatedouro[(integrador,dia)]['mediaMaxIdadeCiclo']:
                cstr_name = f'idade_max_l_{idGalpao}_dia_{dia}_abat_{integrador}'
                solver.Add(var == 0, cstr_name)

        # Zerando variáveis em dias_parado
        for integrador in todos_abatedouros:
            for idGalpao, _ in tempos_by_lote.items():
                granja = lot_to_granja[idGalpao]
                nucleo = lot_to_nucleo[idGalpao]
                zerando_variaveis_dias_parado(solver,dias_parado, idGalpao, granja, nucleo, integrador, venda_lote)

        # definindo lotes em dias fixos
        for idGalpao in inputPersonalizacao["lotes_em_dias_fixos"]:
            lotes_em_dias_fixos(solver,inputPersonalizacao["lotes_em_dias_fixos"], todos_tempos, todos_abatedouros, idGalpao, lot_to_granja, lot_to_nucleo, venda_lote)
        # restrições da quantidade de venda e proporção entre sexos
        for integrador in todos_abatedouros:
            for dia in dias_de_abate[integrador]:

                # restrições para venda máxima e mínima
                venda_minima = dados_abatedouro[(integrador,dia)]['abateMinDiario']
                venda_min_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                                todos_lotes, venda_minima, venda_lote)


                venda_maxima = dados_abatedouro[(integrador,dia)]['abateMaxDiario']
                venda_max_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                            todos_lotes, venda_maxima,venda_lote)


                # restrição quanto a proporção de macho e femea
                if len(lotes_macho) > 0 and len(lotes_femea) > 0:
                    proporcao_max = dados_abatedouro[(integrador,dia)]['proporcaoMaxMachosFemeas']
                    proporcao_maxima_lotes_macho_femea_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                        todos_lotes, lotes_macho,venda_lote, proporcao_max)

                    proporcao_min = dados_abatedouro[(integrador,dia)]['proporcaoMinMachosFemeas']
                    proporcao_minima_lotes_macho_femea_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                        todos_lotes, lotes_macho,venda_lote, proporcao_min)

                    proporcao_max = dados_abatedouro[(integrador,dia)]['proporcaoMaxMachosFemeas']
                    proporcao_maxima_lotes_macho_femea_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                        todos_lotes, lotes_femea,venda_lote, proporcao_max)

                    proporcao_min = dados_abatedouro[(integrador,dia)]['proporcaoMinMachosFemeas']
                    proporcao_minima_lotes_macho_femea_por_dia(solver, dia, integrador, n_animais, lot_to_granja, lot_to_nucleo,
                        todos_lotes, lotes_femea,venda_lote, proporcao_min)

        # ligando venda de lote com venda da granja
        for (idGalpao, (granja, nucleo), dia, integrador), _ in venda_lote.items():
            cstr_name = f'granja_{granja}_lote_{idGalpao}_{dia}_{integrador}'
            solver.Add(
                venda_granja[granja, dia, integrador] >= venda_lote[idGalpao, (granja, nucleo), dia, integrador], cstr_name
            )

        # setando quantidades de dias consecutivos para cada granja
        dias_consecutivos_granja: Dict[str, int] = {granja: 2 for granja in granjas} # esse dado pode ser um input já setado pelo alojamento
        for granja in inputPersonalizacao["granjas_grandes"]:
            if granja in granjas:
                dias_consecutivos_granja[granja] = 3
            # if granja == '301':
            #     dias_consecutivos_granja[granja] = 5

        for integrador in todos_abatedouros:
            for granja in granjas:
                for dia_1 in todos_tempos:
                    for dia_2 in todos_tempos:
                        if dia_2 <= dia_1:
                            continue
                        delta_days = (dia_2 - dia_1).days

                        if delta_days < dias_consecutivos_granja[granja]:
                            continue
                        cstr_name = f'granja_{granja}_dia_{dia_1}_AND_{dia_2}_NOT_{integrador}'
                        solver.Add(
                            venda_granja[granja, dia_1, integrador] + venda_granja[granja, dia_2, integrador] <= 1, cstr_name
                        )

        if inputPersonalizacao["nucleo"]:
            # ligando venda de lote com venda da granja, nucleo
            for (idGalpao, (granja, nucleo), dia, integrador), _ in venda_lote.items():
                cstr_name = f'granja_{granja}_nucleo_{nucleo}_lote_{idGalpao}_{dia}_{integrador}'
                #print((lote,granja,nucleo,dia,integrador),cstr_name)
                solver.Add(
                    venda_nucleo[(granja, nucleo), dia, integrador] >= venda_lote[idGalpao, (granja, nucleo), dia, integrador], cstr_name
                )

            # quantos dias consecutivos podem? (nucleo)
            for integrador in todos_abatedouros:
                for granja in granjas:
                    if granja not in granjas:
                        continue
                    for nucleo in nucleos_by_granja[granja]:
                        for dia_1 in todos_tempos:
                            for dia_2 in todos_tempos:
                                if dia_2 <= dia_1:
                                    continue
                                # print(inputPersonalizacao["dias_consec_granja_nucleo"].keys())
                                if granja in inputPersonalizacao["dias_consec_granja_nucleo"].keys():
                                    dias_cons_usado = inputPersonalizacao["dias_consec_granja_nucleo"][granja]
                                else:
                                    dias_cons_usado = inputPersonalizacao['dias_consec_nucleo_geral']

                                delta_days = (dia_2 - dia_1).days
                                if delta_days < dias_cons_usado:
                                    continue
                                cstr_name = f'granja_{granja}_nucleo_{nucleo}_dia_{dia_1}_AND_{dia_2}_NOT_{integrador}'
                                solver.Add(
                                    venda_nucleo[(granja, nucleo), dia_1, integrador] + venda_nucleo[(granja, nucleo), dia_2, integrador] <= 1, cstr_name
                                )


In [16]:
def solver_resolver():
        global solver, venda_lote, solverParams
        infos = []
        #exit()
        status = solver.Solve(solverParams)

        if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
            print(f'Funcao objetivo ={solver.Objective().Value()}')
            print(solver.WallTime())
            for k, var in venda_lote.items():
                idGalpao, (granja, nucleo), dia, integrador = k
                if venda_lote[k].solution_value() == 1:
                    print(nroLote[k],"-",dia)
                    infos.append([
                        str(idGalpao),
                        int(nroLote[k]),
                        dia,
                        # granja,
                        # nucleo,
                        # aviario[k],
                        idade[k],
                        n_animais[k],
                        round(peso[k],4),
                        round(consumo[k],4),
                        # km[k],
                        round(novo_lucro[k],4),
                        # sexo[k],
                        round(novo_custo[k],4),
                        integrador
                    ])

            return infos
        else:
            print('Nao resolveu o problema')
            return None

In [17]:
def optimize():

        config_solver()
        # print(list_pesos_df)
        # aux=0
        #Roda o otimizador para cada semana
        pesos_df = list_pesos_df
        # print(pesos_df)
        # aux = aux+1
        # print('Rodadas otimizador: ', aux)
        if isinstance( f:= get_values(pesos_df), str):
            print('')

        get_params()
        set_variables_optimizer()
        get_objective_value()
        optimization_rules()

        if not isinstance(f := solver_resolver(), type(None)):
            pd.DataFrame(f).to_csv(f"/content/drive/MyDrive/Otimizador PLT/otimização.csv", sep=";", decimal=",")

In [18]:
optimize()

lucro
Funcao objetivo =147904710979.01605
65142
84117 - 2024-10-15
84146 - 2024-10-14
84127 - 2024-10-14
83813 - 2024-10-11
84123 - 2024-10-12
84145 - 2024-10-15
84031 - 2024-10-07
84116 - 2024-10-15
83741 - 2024-10-08
84097 - 2024-10-10
84053 - 2024-10-07
83814 - 2024-10-11
84032 - 2024-10-08
84033 - 2024-10-09
84111 - 2024-10-11
84099 - 2024-10-10
84025 - 2024-10-08
84148 - 2024-10-15
84105 - 2024-10-10
83826 - 2024-10-10
84098 - 2024-10-10
84512 - 2024-10-16
84118 - 2024-10-16
84095 - 2024-10-15
84027 - 2024-10-09
84039 - 2024-10-07
83752 - 2024-10-08
84054 - 2024-10-09
84149 - 2024-10-15
84093 - 2024-10-15
84531 - 2024-10-15
84079 - 2024-10-11
83828 - 2024-10-10
84124 - 2024-10-12
84519 - 2024-10-14
84126 - 2024-10-12
84021 - 2024-10-07
83742 - 2024-10-07
84037 - 2024-10-08
84159 - 2024-10-16
84080 - 2024-10-11
84091 - 2024-10-15
84022 - 2024-10-09
83827 - 2024-10-10
84113 - 2024-10-16
83816 - 2024-10-12
83751 - 2024-10-07
84143 - 2024-10-15
84119 - 2024-10-12
84052 - 2024-10-07
84